<div>
    <img align="left" src="http://www.twistsystems.com/static/twist/img/twist_logo.png">
</div>
<h1>Exercício de Qualidade de Dados</h1>
<br>
Este projeto foi criado por <b>Grazielle Alessa</b> para o Processo Seletivo da Twist para a vaga de Backend.


In [26]:
import numpy as np
import pandas as pd
import math as mat
import string
from scipy.stats import mode
import pylab as plt
from fancyimpute import MICE
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import skew
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LassoCV
from sklearn.ensemble import IsolationForest
import xgboost as xgb
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn_pandas import DataFrameMapper
import xgboost as xgb
from matplotlib.backends.backend_pdf import PdfPages
import datetime
from sklearn.cluster import FeatureAgglomeration
import seaborn as sns
#Importar os Dados
df = pd.read_excel("database_twist.xlsx")
df

ImportError: No module named fancyimpute

<h2>Exercício 1 - Notas</h2>
Neste exercício, o foco está em criar a matriz de notas. Começando com uma multi-indexação para criar as três colunas de qualidades: Completude, Integridade e Acurácia; Utilizando funções específicas para avaliar cada uma das qualidades e iteração do dataframe original.

In [2]:
#Criacao da Tabela de Notas
qColunas = len(df.columns)
qLinhas = len(df.index)
dfNotas = np.ones((qLinhas,0),dtype=np.int)
dfN = pd.DataFrame(dfNotas)
for char in string.ascii_uppercase:
    colunas = pd.MultiIndex.from_tuples([(char,'Completude'),(char,'Acuracia'),(char,'Integridade')],
                                   names=['lvl0', 'lvl1']) 
    dfNF = pd.DataFrame(dfN,columns=colunas).sort_index().sort_index(axis=1)
    dfN = pd.concat([dfN,dfNF],axis=1)
dfN

,"(A, Acuracia)","(A, Completude)","(A, Integridade)","(B, Acuracia)","(B, Completude)","(B, Integridade)","(C, Acuracia)","(C, Completude)","(C, Integridade)","(D, Acuracia)",...,"(W, Integridade)","(X, Acuracia)","(X, Completude)","(X, Integridade)","(Y, Acuracia)","(Y, Completude)","(Y, Integridade)","(Z, Acuracia)","(Z, Completude)","(Z, Integridade)"
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#Para verificar a Completude, é preciso verificar se há algum dado nulo. Caso haja, a completude está comprometida.
def dadoAusente(x):
    return(mat.isnan(x))

#Para verificar a Integridade, é preciso verificar se há algum dado que não corresponde ao tipo de dados da coluna.
#Para isso, utilizo o comando abaixo:
colTypes = np.array(df.dtypes)
# print colTypes
#colt = pd.DataFrame(colTypes)
#colt

def verTipo(x):
    if(x==0 or x==1 or x==6):
        return(np.int64)
    else: 
        return(np.float64)
    
#Para verificar a Acurácia, é preciso verificar se os dados de cada coluna estão dentro do limite
lim = np.array([[10,1,1,1,2,1,1,0,1,1,2,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
             [100,2,2,2,7,8,2,7,7,4,8,1,3,3,1,2,1,2,4,4,4,9,13,5,2,2]])

In [4]:
#dLimite = pd.DataFrame(lim)
#dLimite
#Iteração com as Linhas do DataFrame
linhas = qLinhas
for colIndex, colName in enumerate(string.ascii_uppercase):
    col = df[colName]
    for linIndex in range(0, linhas):
        item = col[linIndex]
        dA = dadoAusente(item)
        dfN[(colName,'Completude')][linIndex] = 10 if not dA else 0
        dfN[(colName,'Acuracia')][linIndex] = 10 if item >= lim[0,colIndex] and item <= lim[1,colIndex] else 0
        t = verTipo(colIndex)
        dfN[(colName,'Integridade')][linIndex] = 10 if type(item)==t else 0
dfN

,"(A, Acuracia)","(A, Completude)","(A, Integridade)","(B, Acuracia)","(B, Completude)","(B, Integridade)","(C, Acuracia)","(C, Completude)","(C, Integridade)","(D, Acuracia)",...,"(W, Integridade)","(X, Acuracia)","(X, Completude)","(X, Integridade)","(Y, Acuracia)","(Y, Completude)","(Y, Integridade)","(Z, Acuracia)","(Z, Completude)","(Z, Integridade)"
0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
1,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
2,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,10.0,0.0,0.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
3,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,10.0,0.0,0.0,10.0,0.0,0.0,10.0,0.0,0.0,10.0
4,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
5,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,10.0,0.0,0.0,10.0,10.0,10.0,10.0,0.0,0.0,10.0
6,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
7,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
8,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
9,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,10.0,0.0,0.0,10.0,0.0,0.0,10.0,10.0,10.0,10.0


In [27]:
#Adicionando a Media das colunas
dfN.loc['Nota']= dfN.iloc[:62].mean()
#Adicionando a Media das linhas
dfN['Nota']= dfN.mean(axis=1)

#Truncar em 2 casas decimais
pd.options.display.float_format = '{:,.2f}'.format

#Um Output das Notas - Fiz isso devido a visualização de float
exportarNota = pd.ExcelWriter('notas.xlsx')
dfN.to_excel(exportarNota,'Sheet1')
exportarNota.save()

dfN

,"(A, Acuracia)","(A, Completude)","(A, Integridade)","(B, Acuracia)","(B, Completude)","(B, Integridade)","(C, Acuracia)","(C, Completude)","(C, Integridade)","(D, Acuracia)",...,"(X, Acuracia)","(X, Completude)","(X, Integridade)","(Y, Acuracia)","(Y, Completude)","(Y, Integridade)","(Z, Acuracia)","(Z, Completude)","(Z, Integridade)",Nota
0,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,...,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
1,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,...,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
2,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,...,0.00,0.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,9.23
3,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,...,0.00,0.00,10.00,0.00,0.00,10.00,0.00,0.00,10.00,6.15
4,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,...,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
5,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,...,0.00,0.00,10.00,10.00,10.00,10.00,0.00,0.00,10.00,5.13
6,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,...,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
7,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,...,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
8,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,...,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
9,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,...,0.00,0.00,10.00,0.00,0.00,10.00,10.00,10.00,10.00,6.67


<h2>Visão Gerencial </h2>

In [6]:
notaBD = np.array(dfN.iloc[62,78])
print' Nota =',round(notaBD,2)

 Nota = 9.37


<h2>Visão Especialista</h2>

In [22]:
notaV = np.array(dfN.iloc[qLinhas,0:len(dfN.columns)]) #Pega as notas por coluna.
#A cada 3 colunas, fazemos uma media entre as 3.

vetor = np.zeros((1,26),dtype=float)
vMedia = pd.DataFrame(vetor)
notaLinha = dfN.iloc[62,0:78]

n=0
while(n < 76):
    for k in range(0,26):
        indices = np.array([n,n+1,n+2])
        soma = notaLinha.iloc[indices].sum()
        media = soma/3
        vMedia[k]=media
        n = n+3

# notaVar = pd.DataFrame(vMedia)
print'Visao Especialista: \nNotas:'
vMedia

Visao Especialista: 
Notas:


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,10.00,10.00,9.89,9.78,9.25,9.78,10.00,9.68,9.46,9.46,...,9.03,9.03,9.03,9.03,9.03,9.25,9.03,8.92,9.68,9.57


<h2>Visão Analista</h2>

In [24]:
notaA = np.array(dfN.iloc[0:62,78]) #Pega as notas por Linha
notaA = pd.DataFrame(notaA)
print'Visao Analista: \nNotas:'
notaA.transpose() #para melhor visualização

Visao Analista: 
Notas:


,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,10.00,10.00,9.23,6.15,10.00,5.13,10.00,10.00,10.00,6.67,...,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00


<h2>Exercício 2 - Input de Dados Ausentes</h2>

Neste exercício o foco está em adicionar valores para os dados não informados do dataframe. Para isso, escolherei utilizar a moda dos valores da coluna como candidata a input.

<b>A próxima célula é uma versão alternativa de dar input nos dados ausentes, não utilizando o método exigido.</b>

In [9]:
# Exercício 2 - Input de Dados Ausentes

#Verificando o numero de valores ausentes por coluna
dfConsertado = df
dfConsertado.apply(lambda x: sum(x.isnull()),axis=0)

#Adicionando valores medios
for char in string.ascii_uppercase:
    dfConsertado[char].fillna(round(df[char].mean(),1), inplace=True)
dfConsertado


,A,B,C,D,E,F,G,H,I,J,...,Q,R,S,T,U,V,W,X,Y,Z
0,40,1,2.00,1.00,2.00,3.00,2,2.00,3.00,2.00,...,0.00,1.00,1.00,1.00,1.00,3.00,6.00,3.00,2.00,2.00
1,32,1,2.00,1.00,4.00,1.00,1,6.00,4.00,3.00,...,1.00,0.00,1.00,1.00,1.00,2.00,5.00,2.00,2.00,2.00
2,56,1,2.00,1.00,2.00,2.00,2,0.00,1.00,4.00,...,0.00,0.00,0.00,0.00,0.00,3.20,4.60,1.90,1.00,2.00
3,60,1,2.00,1.00,3.30,1.00,2,3.00,3.00,4.00,...,0.50,0.50,0.70,1.00,1.50,1.00,4.60,1.90,2.00,2.00
4,44,1,2.00,1.00,3.00,7.00,2,0.00,3.00,2.00,...,0.00,1.00,0.00,1.00,2.00,3.00,4.00,2.00,2.00,2.00
5,75,1,2.00,2.00,3.30,1.00,2,2.30,3.40,2.30,...,0.50,0.50,0.70,1.00,1.50,3.20,4.60,1.90,2.00,2.00
6,47,2,2.00,2.00,2.00,2.00,2,3.00,2.00,1.00,...,0.00,0.00,1.00,1.00,1.00,3.00,4.00,1.00,2.00,2.00
7,52,1,2.00,2.00,4.00,1.00,2,7.00,4.00,2.00,...,1.00,1.00,4.00,4.00,4.00,4.00,2.00,2.00,2.00,2.00
8,34,1,2.00,1.00,3.00,3.00,2,2.00,4.00,3.00,...,1.00,0.00,0.00,0.00,1.00,2.00,4.00,1.00,2.00,2.00
9,38,1,2.00,1.00,3.30,3.00,2,2.00,7.00,4.00,...,0.50,0.50,0.70,1.00,1.50,2.00,6.00,1.90,2.00,2.00


<b>OBS: </b>Abaixo está uma tentativa de fazer o input de dados utilizando o <a font-color=blue>statsmodels.imputation.mice.MICEData</a>, mas tive alguns problemas durante a realização deste projeto.
<br>
Entre os problemas encontrados e dificuldades, destaco:
<ul>
<li>Não reconhecimento do import do modulo mice</li>
<li>Tentativa 1: Utilizar script em R para utilizar o pacote, mas o import das bibliotecas que dão suporte aos comandos em R também tiveram problemas na importação</li>
</ul>

In [10]:
# #Verificando o numero de valores ausentes por coluna
# dfCon = df
# dfCon.apply(lambda x: sum(x.isnull()),axis=0)

# # Imputation using MICE
# numerical_features_names = self.extract_numerical_features(dfCon)
# dfCon.loc[:, tuple(numerical_features_names)] = self.estimate_by_mice(
# dfCon[numerical_features_names])